In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split
from scipy.stats import mode
from scipy.spatial import distance as sci_distance

In [2]:
iris = load_iris()
X_iris = iris.data
Y_iris = iris.target

In [3]:
X_iris_train, X_iris_test, Y_iris_train, Y_iris_test = train_test_split(X_iris, Y_iris, test_size = 0.5)

In [4]:
Y_iris_train.shape

(75,)

In [5]:
def euclideanDistance(x_a, x_b):
    """
    Calculates the Euclidean distance between two vectors
    
    Arguments:
        x_a (np.array): shape [m_features, ] a single vector a
        x_b (np.array): shape [m_features, ] a single vector b
    
    Returns:
        distance (float): Euclidean distance between vectors x_a and x_b
    """
    
    return minkowskiDistance(x_a, x_b, 2)

In [6]:
def manhattanDistance(x_a, x_b):
    """
    Calculates the Manhattan distance between two vectors
    
    Arguments:
        x_a (np.array): shape [m_features, ] a single vector a
        x_b (np.array): shape [m_features, ] a single vector b
    
    Returns:
        distance (float): Manhattan distance between vectors x_a and x_b
    """
    return minkowskiDistance(x_a, x_b, 1)

In [7]:
def chebyshevDistance(x_a, x_b):
    """
    Calculates the Chebyshev distance between two vectors
    
    Arguments:
        x_a (np.array): shape [m_features, ] a single vector a
        x_b (np.array): shape [m_features, ] a single vector b
    
    Returns:
        distance (float): Chebyshev distance between vectors x_a and x_b
    """
    
    distance = np.max( np.abs(x_a - x_b) ) 
    return distance

In [8]:
def minkowskiDistance(x_a, x_b, p=2):
    """
    Calculates the minkowski distance between two vectors
    
    Arguments:
        x_a (np.array): shape [m_features, ] a single vector a
        x_b (np.array): shape [m_features, ] a single vector b
        p (int): Sets the Lp distance metric to use:
            1 - Manhattan
            2 - Euclidian 
            inf - Chebyshev
    
    Returns:
        distance (float): Minkowski distance between vectors x_a and x_b
    """
    
    distance = np.sum(np.abs(x_a - x_b)**p)**(1/p)
    return distance

In [9]:
def mahalanobisDistance(x_a, x_b, iCov):
    return sci_distance.mahalanobis(x_a, x_b, iCov)

In [10]:
def idealKernelMatrix(Y_in, l=0.5):
    Y_in_KD  = np.zeros(shape=(Y_in.shape[0], (max(Y_in)+1) ))

    for i in range(Y_in.shape[0]):
        Y_in_KD[i][Y_in[i]] = 1
    Y_in_KD = Y_in_KD.T #In the paper they have m and n swapped
    return (Y_in_KD.T)@(Y_in_KD) + l*np.identity(Y_in_KD.shape[1])    

In [11]:
def optimalDistanceMetric(X_in, Y_in):
    X_in = X_in.T #In the paper they have m and n swapped
    iKD = np.linalg.inv(idealKernelMatrix(Y_in))
    return np.linalg.inv( X_in@iKD@X_in.T )

In [12]:
def optimalDistance(x_a, x_b, A):
    return np.sqrt( (x_a - x_b).T@A@(x_a - x_b) )

In [13]:
def calculateDistances(x_test, X_in, Y_in, distanceFunction):
    """
    Calculates the distance between a single test example, x_test,
    and a list of examples X_in. 
    
    Args:
        x_test (np.array): shape [n_features,] a single test example
        X_in (np.array): shape [n_samples, n_features] a list of examples to compare against.
    
    Returns:
        distance_list (list of float): The list containing the distances       
    """
    distance_list = []
    if distanceFunction == optimalDistance:
        A = optimalDistanceMetric(X_in, Y_in)
        for example in X_in:
            distance_list.append(distanceFunction(example, x_test, A))
        
    elif distanceFunction == mahalanobisDistance:
        iCov = np.linalg.inv(np.cov(X_in, rowvar=False))
        for example in X_in:
            distance_list.append(distanceFunction(example, x_test, iCov))
    else:
        for example in X_in:
            distance_list.append(distanceFunction(example, x_test))
    return distance_list

In [14]:
def kNearestIndices(distance_list, k):
    """
    Determines the indices of the k nearest neighbours
    
    Arguments:
        distance_list (list of float): list of distances between a test point 
            and every training example
        k (int): the number of nearest neighbours to consider
    
    Returns:
        k_nearest_indices (array of int): shape [k,] array of the indices 
            corresponding to the k nearest neighbours
    """
    
    k_nearest_indices = np.array( np.argsort(distance_list)[:k] )
    return k_nearest_indices

In [15]:
def kNearestNeighbours(k_nearest_indices, X_in, Y_in):
    """
    Creates the dataset of k nearest neighbours
    
    Arguments:
        k_nearest_indices (array of int): shape [k,] array of the indices 
            corresponding to the k nearest neighbours
        X_in (array): shape [n_examples, n_features] the example data matrix to sample from
        Y_in (array): shape [n_examples, ] the label data matrix to sample from
    
    Returns:
        X_k (array): shape [k, n_features] the k nearest examples
        Y_k (array): shape [k, ] the labels corresponding to the k nearest examples
    """
    
    X_k = []
    Y_k = []

    for i in k_nearest_indices:
        X_k.append(X_in[i])
        Y_k.append(Y_in[i])
        
    X_k = np.array(X_k)
    Y_k = np.array(Y_k)
    return X_k, Y_k

In [16]:
def predict(x_test, X_in, Y_in, k, distanceFunction):
    """
    Predicts the class of a single test example
    
    Arguments:
        x_test (np.array): shape [n_features, ] the test example to classify
        X_in (np.array): shape [n_input_examples, n_features] the example data matrix to sample from
        Y_in (np.array): shape [n_input_labels, ] the label data matrix to sample from
    
    Returns:
        prediction (array): shape [1,] the number corresponding to the class 
    """
    distance_list = calculateDistances(x_test, X_in, Y_in, distanceFunction)
    kNN_indices = kNearestIndices(distance_list, k)
    X_k, Y_k = kNearestNeighbours(kNN_indices, X_in, Y_in)
    prediction =  mode(Y_k, axis=None)[0]

    return prediction

In [17]:
def predictBatch(X_t, X_in, Y_in, k, distanceFunction):
    """
    Performs predictions over a batch of test examples
    
    Arguments:
        X_t (np.array): shape [n_test_examples, n_features]
        X_in (np.array): shape [n_input_examples, n_features]
        Y_in (np.array): shape [n_input_labels, ]
        k (int): number of nearest neighbours to consider
    
    Returns:
        predictions (np.array): shape [n_test_examples,] the array of predictions
        
    """
    predictions = []
    for x_t_i in X_t:
        predictions.append(predict(x_t_i, X_in, Y_in, k, distanceFunction)[0])
    
    return np.array(predictions)

In [18]:
def accuracy(Y_pred, Y_test):
    """
    Calculates the accuracy of the model 
    
    Arguments:
        Y_pred (np.array): shape [n_test_examples,] an array of model predictions
        Y_test (np.array): shape [n_test_labels,] an array of test labels to 
            evaluate the predictions against
    
    Returns:
        accuracy (float): the accuracy of the model
    """
    assert(Y_pred.shape == Y_test.shape)
    
    correct = 0
    total = len(Y_test)

    for i in range(total):
        if (Y_pred[i] == Y_test[i]):
            correct += 1
    
    accuracy = correct/total
    return accuracy

In [19]:
def run(X_train, X_test, Y_train, Y_test, k, distanceFunction=euclideanDistance):
    """
    Evaluates the model on the test data
    
    Arguments:
        X_train (np.array): shape [n_train_examples, n_features]
        X_test (np.array): shape [n_test_examples, n_features]
        Y_train (np.array): shape [n_train_examples, ]
        Y_test (np.array): shape [n_test_examples, ]
        k (int): number of nearest neighbours to consider
    
    Returns:
        test_accuracy (float): the final accuracy of your model 
    """
    Y_pred = predictBatch(X_test, X_train, Y_train, k, distanceFunction)
    test_accuracy = accuracy(Y_pred, Y_test)

    return test_accuracy

In [20]:
print( run(X_iris_train, X_iris_test, Y_iris_train, Y_iris_test, 4, manhattanDistance) ) 

0.9866666666666667


In [21]:
print( run(X_iris_train, X_iris_test, Y_iris_train, Y_iris_test, 4, chebyshevDistance) ) 

0.9866666666666667


In [22]:
print( run(X_iris_train, X_iris_test, Y_iris_train, Y_iris_test, 4, euclideanDistance) ) 

0.9866666666666667


In [23]:
print( run(X_iris_train, X_iris_test, Y_iris_train, Y_iris_test, 4, mahalanobisDistance) ) 

(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
0.8933333333333333


In [24]:
print( run(X_iris_train, X_iris_test, Y_iris_train, Y_iris_test, 4, optimalDistance) ) 

(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
(4, 4)
0.9733333333333334


In [25]:
row1 = np.array([10, 20, 15, 10, 5])
row2 = np.array([12, 24, 18, 8, 7])
print(chebyshevDistance(row1, row2))

4


In [26]:
row1 = np.array([0, 3, 4, 5])
row2 = np.array([7, 6, 3, -1])
print(chebyshevDistance(row1, row2))

7


In [27]:
X_iris_train

array([[7.7, 2.6, 6.9, 2.3],
       [5.5, 2.5, 4. , 1.3],
       [6. , 2.2, 5. , 1.5],
       [6.9, 3.2, 5.7, 2.3],
       [6.4, 2.8, 5.6, 2.2],
       [6.7, 3.3, 5.7, 2.1],
       [7.1, 3. , 5.9, 2.1],
       [5.2, 2.7, 3.9, 1.4],
       [4.5, 2.3, 1.3, 0.3],
       [5.1, 3.4, 1.5, 0.2],
       [6. , 3. , 4.8, 1.8],
       [6.5, 3.2, 5.1, 2. ],
       [6.1, 2.8, 4.7, 1.2],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3. , 1.6, 0.2],
       [6.2, 2.2, 4.5, 1.5],
       [5. , 3.5, 1.6, 0.6],
       [6.8, 2.8, 4.8, 1.4],
       [4.4, 3. , 1.3, 0.2],
       [6.3, 2.5, 5. , 1.9],
       [5.4, 3.4, 1.5, 0.4],
       [6.2, 2.9, 4.3, 1.3],
       [6.3, 2.8, 5.1, 1.5],
       [6.4, 3.1, 5.5, 1.8],
       [6.4, 3.2, 5.3, 2.3],
       [5.5, 3.5, 1.3, 0.2],
       [5.7, 3.8, 1.7, 0.3],
       [6.4, 2.9, 4.3, 1.3],
       [6.4, 2.7, 5.3, 1.9],
       [6.6, 2.9, 4.6, 1.3],
       [5.5, 2.6, 4.4, 1.2],
       [5. , 3.3, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [6.7, 2.5, 5.8, 1.8],
       [5. , 3

In [28]:
print(Y_iris_train)

[2 1 2 2 2 2 2 1 0 0 2 2 1 0 0 1 0 1 0 2 0 1 2 2 2 0 0 1 2 1 1 0 0 2 0 2 0
 2 1 1 2 1 0 0 0 1 2 2 0 2 1 0 0 1 2 1 1 2 1 0 2 1 1 2 0 0 0 1 2 1 1 0 1 1
 0]


In [29]:
Y_in_KD  = np.zeros(shape=(Y_iris_train.shape[0], (max(Y_iris_train)+1) ))

for i in range(X_iris_train.shape[0]):
    Y_in_KD[i][Y_iris_train[i]] = 1
    
Y_in_KD = Y_in_KD.T
print(Y_in_KD.shape)

(3, 75)


In [30]:
print(idealKernelMatrix(Y_iris_train, l=0.3).shape)

(75, 75)


In [31]:
Y_in_KD@Y_in_KD.T #TODO THE DIMS FIXXX

array([[25.,  0.,  0.],
       [ 0., 25.,  0.],
       [ 0.,  0., 25.]])

In [32]:
(Y_in_KD@Y_in_KD.T).shape

(3, 3)

In [33]:
print(idealKernelMatrix(Y_iris_train, l=0.3))

[[1.3 0.  1.  ... 0.  0.  0. ]
 [0.  1.3 0.  ... 1.  1.  0. ]
 [1.  0.  1.3 ... 0.  0.  0. ]
 ...
 [0.  1.  0.  ... 1.3 1.  0. ]
 [0.  1.  0.  ... 1.  1.3 0. ]
 [0.  0.  0.  ... 0.  0.  1.3]]


In [34]:
print(optimalDistanceMetric(X_iris_train, Y_iris_train))

(4, 4)
[[ 0.0692136  -0.0521936  -0.06483095  0.02282675]
 [-0.0521936   0.08897734  0.03411109 -0.04800189]
 [-0.06483095  0.03411109  0.1031973  -0.0934645 ]
 [ 0.02282675 -0.04800189 -0.0934645   0.29204287]]


In [35]:
print(Y_iris_train.shape)

(75,)
